In [62]:
from FramesAtGivenScaledImage import FramesAtGivenScaledImage

In [88]:
from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms

#personal imports
from configuration import *
from CustomDatasetFromImages import CustomDatasetFromImages
from cut_image import Chunking
from detections import *
from DetectionCandidate import *
from FramesAtGivenScaledImage import FramesAtGivenScaledImage
from ImageWithDetections import ImageWithDetections
import Net

In [2]:
def train_neural_network(train_loader, net, optimizer, criterion):
    print('Start training')
    for epoch in range(2):  # loop over the dataset multiple times

        running_loss = 0.0
        for i, data in enumerate(train_loader, 0):
            inputs, labels = data

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
            if i % 2000 == 1999:  # print every 2000 mini-batches
                print('[%d, %5d] loss: %.3f' %
                      (epoch + 1, i + 1, running_loss / 2000))
                running_loss = 0.0

    print('Finished Training')
    return net

In [37]:
def test_neural_network_original_dataset(test_loader, net):
    class_correct = list(0. for i in range(2))
    class_total = list(0. for i in range(2))
    classes = (0, 1,)
    with torch.no_grad():
        for data in test_loader:
            images, labels = data
            labels = torch.LongTensor(([1 if label > 1 else label for label in labels ]))
            outputs = net(images)
            _, predicted = torch.max(outputs, 1)
            c = (predicted == labels).squeeze()
            for i in range(len(labels)):
                label = labels[i]
                class_correct[label] += c[i].item()
                class_total[label] += 1

    for i in range(2):
        print('Accuracy of %5s : %2d %%' % (
            classes[i], 100 * class_correct[i] / class_total[i]))

    correct = 0
    total = 0
    with torch.no_grad():
        for data in test_loader:
            images, labels = data
            labels = torch.LongTensor(([1 if label > 1 else label for label in labels ]))
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the 10000 test images: %d %%' % (
            100 * correct / total))

In [89]:
def test_neural_network_own_dataset(path_to_image, net):
    im = Image.open(path_to_image)
    # Convert into grayscale
    #im = im.convert("L")

    #cut image to different scales
    Chunking.init(im)
    (scaled_images, scaled_positions) = Chunking.get_imgchunks_atdiffscales(strides=(5, 5), nbshrinkages=3, divfactor=2)

    #compute score for each frame
    frames = []
    scaled_factor = 1
    for index in range(len(scaled_images)):
        print(scaled_factor)
        dataset = CustomDatasetFromImages(scaled_images[index])
        ##test on neural network
        #get the dataloader. Minibatches of 4
        dataloader = torch.utils.data.DataLoader(dataset, batch_size=4,
                                                 shuffle=True, num_workers=2)
        scores = []
        with torch.no_grad():
            for data in dataloader:
                images, _ = data
                outputs = net(images) #[(score of class_0, score of class_1), ...]
                outputs = outputs.numpy()
                outputs = outputs.tolist()
                scores.extend(outputs)
        frames.append(FramesAtGivenScaledImage(scaled_factor, dataset, scaled_positions[index], scores))
        scaled_factor = division_factor*(2**index)

    #subdetections
    subdetections = capture_good_positions(frames)
    print("subdetections:")
    for subd in subdetections:
        print(subd)

    """3/ Clustering of DetectionCandidates into Detections and filtering"""
    detections = getDetections(subdetections, min_samples=2)  # min_samples --> number of min detection to confirm the detection
    print(detections)  # In the example, one subdetection is alon in a detection => this detection is discarded.

    """4/ Get the best candidates"""
    winners = get_best_clusters_candidates(subdetections, detections)
    for w in winners:
        print(w)

    """5/ Save an image with all subdetections, and then only with the kept subdetections"""

    # With all subdetections
    imdet = ImageWithDetections(im, subdetections)
    original_name_image = path_to_image.split('.')
    imdet.save(original_name_image[0]+"_all_detections.JPG")

    # With only winner subdetections
    imdet.subdetections = winners
    imdet.save(original_name_image[0]+"_winners_detections.JPG")

In [5]:
#training data set
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize([0.5], [0.5])])
train_imagenet = torchvision.datasets.ImageFolder('start_deep/start_deep/start_deep/train_images/',
                                                  transform=transform)
train_loader = torch.utils.data.DataLoader(train_imagenet, batch_size=4, shuffle=True, num_workers=2)

#define net and define optimizer
net = Net.Net()
weights = torch.Tensor([1, 0.5])
criterion = nn.CrossEntropyLoss(weights)
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [6]:
#training
net = train_neural_network(train_loader, net, optimizer, criterion)

Start training
[1,  2000] loss: 0.388
[1,  4000] loss: 0.165
[1,  6000] loss: 0.127
[1,  8000] loss: 0.114
[1, 10000] loss: 0.091
[1, 12000] loss: 0.081
[1, 14000] loss: 0.067
[1, 16000] loss: 0.058
[1, 18000] loss: 0.048
[1, 20000] loss: 0.052
[1, 22000] loss: 0.041
[2,  2000] loss: 0.037
[2,  4000] loss: 0.029
[2,  6000] loss: 0.032
[2,  8000] loss: 0.029
[2, 10000] loss: 0.031
[2, 12000] loss: 0.028
[2, 14000] loss: 0.019
[2, 16000] loss: 0.019
[2, 18000] loss: 0.022
[2, 20000] loss: 0.016
[2, 22000] loss: 0.022
Finished Training


In [38]:
#test with original dataset
test_imagenet = torchvision.datasets.ImageFolder('start_deep/start_deep/start_deep/test_images/',
                                                 transform=transform)
test_loader = torch.utils.data.DataLoader(test_imagenet, batch_size=4, shuffle=True, num_workers=2)
test_neural_network_original_dataset(test_loader, net)

Accuracy of     0 : 98 %
Accuracy of     1 : 31 %
Accuracy of the network on the 10000 test images: 91 %


In [90]:
path_to_image = input("(ex : own_dataset/ex3.JPG). The path to the image to test is :")
test_neural_network_own_dataset(path_to_image, net)

(ex : own_dataset/ex3.JPG). The path to the image to test is :own_dataset/ex5.JPG
1
1.2
2.4
4.8
subdetections:
score:7.071359157562256; position:(0, 35); dims:(36, 36)
score:1.894158661365509; position:(0, 115); dims:(36, 36)
score:1.1626574993133545; position:(5, 30); dims:(36, 36)
score:19.297537803649902; position:(10, 40); dims:(36, 36)
score:1.6784915924072266; position:(10, 85); dims:(36, 36)
score:0.9707764983177185; position:(10, 265); dims:(36, 36)
score:1.1851617693901062; position:(10, 305); dims:(36, 36)
score:2.1951162815093994; position:(15, 35); dims:(36, 36)
score:5.585206031799316; position:(15, 324); dims:(36, 36)
score:0.33121080696582794; position:(20, 75); dims:(36, 36)
score:0.997149646282196; position:(20, 90); dims:(36, 36)
score:1.151901662349701; position:(20, 320); dims:(36, 36)
score:0.30714984238147736; position:(25, 75); dims:(36, 36)
score:17.428054809570312; position:(25, 115); dims:(36, 36)
score:10.211194038391113; position:(30, 5); dims:(36, 36)
score

score:9.81891918182373; position:(305, 155); dims:(36, 36)
score:0.9079889357089996; position:(305, 180); dims:(36, 36)
score:2.7703839540481567; position:(305, 230); dims:(36, 36)
score:4.492739915847778; position:(305, 255); dims:(36, 36)
score:11.299095630645752; position:(305, 315); dims:(36, 36)
score:9.352459907531738; position:(310, 310); dims:(36, 36)
score:8.27160096168518; position:(315, 55); dims:(36, 36)
score:13.870831489562988; position:(315, 280); dims:(36, 36)
score:2.060836434364319; position:(324, 15); dims:(36, 36)
score:4.639812231063843; position:(324, 85); dims:(36, 36)
score:0.18997175991535187; position:(324, 110); dims:(36, 36)
score:2.286090672016144; position:(324, 125); dims:(36, 36)
score:4.079974293708801; position:(0, 96); dims:(43, 43)
score:2.6338096857070923; position:(0, 102); dims:(43, 43)
score:1.9276899099349976; position:(0, 144); dims:(43, 43)
score:3.362184762954712; position:(6, 36); dims:(43, 43)
score:2.6540225744247437; position:(6, 48); dim